In [ ]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer
 
model_path = r"THUDM/chatglm2-6b-int4"

#fill in here
prefix_state_dict = r"/kaggle/input/{your-dataset-name}/{your-model}.bin"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained(model_path, config=config, trust_remote_code=True)

prefix_state_dict = torch.load(prefix_state_dict,map_location=torch.device('cpu'))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)


model = model.quantize(4)
model = model.float()
model.transformer.prefix_encoder.float()


model = model.eval()
print('total params: ', sum(p.numel() for p in model.parameters()))

In [ ]:
response, history = model.chat(tokenizer, "你好!", history=[])
print(response)
print(history)
response, history = model.chat(tokenizer, "你现在开心吗", history=[])
print(response)
print(history)
response, history = model.chat(tokenizer, "你在干什么", history=[])
print(response)
print(history)


In [ ]:
response, history = model.chat(tokenizer, "你最喜欢什么", history=[])
print(response)
print(history)
response, history = model.chat(tokenizer, "你明天想出去玩吗", history=[])
print(response)
print(history)
response, history = model.chat(tokenizer, "什么时候有空", history=[])
print(response)
print(history)